## what to do next

class bitmexDataHandler
- bm_ws_market  订阅 成交明细，买一卖一
- snapshot()   500毫秒一个切片，发送MarketEvent

其中bm_ws_market中的如下成员有用：
- prev_bar
- current_bar
- bid1, ask1

todo:
1. utils.calc_ts()
2. 发布订阅 bar_type, bm_ws_market.subscribe('XBTUSD', ['1m', '5m'], false)
3. bar和last_price需要有两个时间戳：交易所时间和本机实际收到的时间
4. 定义一些全局变量，eg. 枚举类 BarType, def n_minutes()获取对应的分钟数

bar_type 可以拼成任何时间周期，eg. 9m, 17m, 18m, 可以有shift   
所以bar_type的格式是这样的： '9m3s', '18m25s'   
trading_day, ts = 20180101, 190015   
目前可以实现的： trading_day, ts = calc_td_ts()， ts保留至秒钟

## next (Otc 7th)

## next: 路线图

1. 切换回 master 分支
2. 重新组织结构：
    + 将DataHandler中的MarketData定义剥离出来
    + 主目录下文件清理
3. 定义事件，写事件引擎
    + event/eventType.py, 
    + evetn/eventEvgine.py
4. 将 bitmex/main.py 封装成 MainEngine
5. MainEngine, EventEngine与`DataHandler`联调
6. 写Strategy
7. `Strategy`, DataHandler 与 EventEngine 联调
8. 写Portfolio
9. `Protoflio`, Strategy, DataHandler 与 EventEngine 联调

![](figs/structure.jpeg)